In [2]:
from all_imports import *
from pdb import set_trace
from tabulate import tabulate
import scikit_posthocs as sp
import pingouin as pg
import warnings
warnings.filterwarnings('ignore')
from getdata import DataSlicer


### Reads dataset

In [3]:
# Create an instance of the DataSlicer class
data_path = r"C:\phaseII\static\dataset-individual-Items.csv"
data_slicer = DataSlicer(data_path)
# Call the functions on the data_slicer instance
df = data_slicer.read_data()
data= data_slicer.average_scores()
# Convert the data types of the columns to float
data = data.iloc[:, -10:].astype(float)
data.tail()

,Social proof,Flattery,Rht_Question,Antanagoge,Logic,Authority,Pathos,Repetition,Framing,Anaphora
1990,3.000000,7.000000,3.000000,6.000000,3.000000,9.666667,7.666667,3.333333,6.666667,6.666667
1991,4.666667,8.000000,5.333333,1.333333,2.333333,8.333333,6.333333,4.666667,7.666667,7.666667
1992,5.333333,6.000000,6.666667,6.333333,5.666667,6.333333,5.333333,4.666667,5.666667,5.666667
1993,6.333333,8.333333,6.666667,5.666667,6.333333,9.333333,6.666667,6.333333,6.333333,8.333333
1994,4.333333,4.000000,4.666667,5.666667,6.666667,5.333333,4.666667,5.666667,5.000000,5.333333


### The Friedman test the non-parametric alternative to ANOVA

####  Distribution of Influence Scores of the Average Persuasion Goal

In [ ]:
### Shuffle the DataFrame

### Friedman test

In [11]:
from tabulate import tabulate
from itertools import combinations
from pdb import set_trace
# Perform the Friedman test
stat, p = friedmanchisquare(*[data[col] for col in data.columns])

alpha = 0.05
k = 10  # Number of tests (persuasion techniques)
bonf_alpha = alpha / k

print(f"Friedman Test: Test Statistic = {stat:.3f}, p-value = {p:.3f}")




# Check if the difference is significant after Bonferroni correction
if p <= bonf_alpha:
    print("The difference is statistically significant.")
else:
    print("The difference is not statistically significant.")

# Create a table to display the results
headers = ['Test', 'Value']
table_data = [['Friedman test (Chi-squared)', '{:.4f}'.format(stat)],
              ['Friedman test (Degrees of freedom)', '{}'.format(len(data.columns)-1)],
              ['Friedman test (P value)', '{:.4f}'.format(p)],
              ['Bonferroni-adjusted significance level', '{:.4f}'.format(bonf_alpha)],
              ['Significance after Bonferroni correction', 'Yes' if p <= bonf_alpha else 'No']]

table = tabulate(table_data, headers=headers, tablefmt="fancy_grid")
print(table)




Friedman Test: Test Statistic = 1822.439, p-value = 0.000
The difference is statistically significant.
╒══════════════════════════════════════════╤═══════════╕
│ Test                                     │ Value     │
╞══════════════════════════════════════════╪═══════════╡
│ Friedman test (Chi-squared)              │ 1822.4390 │
├──────────────────────────────────────────┼───────────┤
│ Friedman test (Degrees of freedom)       │ 9         │
├──────────────────────────────────────────┼───────────┤
│ Friedman test (P value)                  │ 0.0000    │
├──────────────────────────────────────────┼───────────┤
│ Bonferroni-adjusted significance level   │ 0.0050    │
├──────────────────────────────────────────┼───────────┤
│ Significance after Bonferroni correction │ Yes       │
╘══════════════════════════════════════════╧═══════════╛


In [13]:

# Bonferroni correction
alpha = 0.05
k = len(list(combinations(data.columns, 2)))  # Use the number of pairwise comparisons
bonf_alpha = alpha / k  # original_significance_level / number_of_tests.

wilcoxon_results = []
for col1, col2 in combinations(data.columns, 2):
    statistic, p_value = wilcoxon(data[col1], data[col2])  # Calculates the test statistic (`statistic`) and the corresponding p-value (`p_value`).

    adjusted_p_value = min(p_value * k, 1.0)  # Bonferroni-adjusted p-value (bounded between 0 and 1)
    reject_null_pair = adjusted_p_value <= alpha

    # Determine the direction of the difference
    if reject_null_pair:
        if data[col1].mean() > data[col2].mean():
            direction = f"{col1} > {col2}"
        else:
            direction = f"{col2} > {col1}"
    else:
        direction = "-"

    wilcoxon_results.append((col1, col2, statistic, p_value, adjusted_p_value, reject_null_pair, direction))

# Create a table to display the results
headers = ['Test', 'TS', 'P-value', 'Adj. P-value', 'Significance', 'Direction']
table_data = []
for result in wilcoxon_results:
    pair = f"{result[0]} vs {result[1]}"
    stat = '{:.5f}'.format(result[2])
    p_value = '{:.5f}'.format(result[3])
    adjusted_p_value = '{:.5f}'.format(result[4])
    significant = 'Yes' if result[5] else 'No'
    direction = result[6]
    table_data.append([pair, stat, p_value, adjusted_p_value, significant, direction])

table = tabulate(table_data, headers=headers, tablefmt="fancy_grid")
print('')
print(f"Bonferroni-adjusted Significance Level: {bonf_alpha}")
print('')
print(table)



Bonferroni-adjusted Significance Level: 0.0011111111111111111

╒══════════════════════════════╤════════╤═══════════╤════════════════╤════════════════╤═════════════════════════════╕
│ Test                         │     TS │   P-value │   Adj. P-value │ Significance   │ Direction                   │
╞══════════════════════════════╪════════╪═══════════╪════════════════╪════════════════╪═════════════════════════════╡
│ Social proof vs Flattery     │ 527893 │   0       │        0       │ Yes            │ Flattery > Social proof     │
├──────────────────────────────┼────────┼───────────┼────────────────┼────────────────┼─────────────────────────────┤
│ Social proof vs Rht_Question │ 721488 │   5e-05   │        0.00209 │ Yes            │ Rht_Question > Social proof │
├──────────────────────────────┼────────┼───────────┼────────────────┼────────────────┼─────────────────────────────┤
│ Social proof vs Antanagoge   │ 721420 │   0       │        3e-05   │ Yes            │ Antanagoge > Social pr

In [32]:
wilcoxon_results

[('Social proof',
  'Flattery',
  527893.0,
  5.750197152677329e-39,
  2.587588718704798e-37,
  True,
  'Flattery > Social proof'),
 ('Social proof',
  'Rht_Question',
  721488.5,
  4.639369179238244e-05,
  0.0020877161306572098,
  True,
  'Rht_Question > Social proof'),
 ('Social proof',
  'Antanagoge',
  721420.5,
  5.745509554360448e-07,
  2.5854792994622014e-05,
  True,
  'Antanagoge > Social proof'),
 ('Social proof',
  'Logic',
  526180.0,
  4.259225087067594e-40,
  1.9166512891804173e-38,
  True,
  'Logic > Social proof'),
 ('Social proof',
  'Authority',
  395295.0,
  2.463966770909988e-85,
  1.1087850469094946e-83,
  True,
  'Authority > Social proof'),
 ('Social proof',
  'Pathos',
  417386.0,
  8.77689648119919e-76,
  3.9496034165396355e-74,
  True,
  'Pathos > Social proof'),
 ('Social proof',
  'Repetition',
  722215.0,
  1.7426310715982187e-06,
  7.841839822191984e-05,
  True,
  'Repetition > Social proof'),
 ('Social proof',
  'Framing',
  138415.0,
  1.0816266934535137e